In [25]:

import pandas as pd
from itertools import combinations
import numpy as np

# Load the Excel file
file_path = '/home/abdelrahman.sadallah/mbzuai/review_rewrite/chatgpt/outputs/main_data_batch_gold_results.xlsx'
import pandas as pd
import numpy as np
import itertools

# Load Excel file
aspects = ['actionability', 'grounding_specificity', 'verifiability', 'helpfulness']
sheets = {aspect: pd.read_excel(file_path, sheet_name=aspect) for aspect in aspects}

# Step 1: Get common review_points across all sheets
common_review_points = set(sheets[aspects[0]]['review_point'])
for aspect in aspects[1:]:
    common_review_points &= set(sheets[aspect]['review_point'])

# Step 2: Filter to rows with common review_points
for aspect in aspects:
    sheets[aspect] = sheets[aspect][sheets[aspect]['review_point'].isin(common_review_points)]

# Step 3: Merge all sheets on 'review_point'
merged_df = sheets[aspects[0]][['review_point', 'venue', f'{aspects[0]}_label', f'chatgpt_{aspects[0]}_score']]
for aspect in aspects[1:]:
    merged_df = pd.merge(
        merged_df,
        sheets[aspect][['review_point', f'{aspect}_label', f'chatgpt_{aspect}_score']],
        on='review_point',
        how='inner'
    )

# Step 4: Normalize venue (e.g. ICLR_2024 → ICLR)
merged_df['main_venue'] = merged_df['venue'].apply(lambda v: v.split('_')[0] if isinstance(v, str) else v)

# Step 5: Preprocess verifiability: replace 'X' with 0 and cast all labels to float
label_cols = [f"{aspect}_label" for aspect in aspects]
merged_df['verifiability_label'] = merged_df['verifiability_label'].replace('X', 0)

for col in label_cols:
    merged_df[col] = pd.to_numeric(merged_df[col], errors='coerce')

# Step 6: Compute custom variance as sum of absolute pairwise differences
def custom_score_diversity(row):
    values = [row[col] for col in label_cols if pd.notnull(row[col])]
    return sum(abs(a - b) for a, b in itertools.combinations(values, 2))

merged_df['score_diversity'] = merged_df.apply(custom_score_diversity, axis=1)

# Step 7: Select one low-diversity and one high-diversity example per main venue (ensuring true diversity)

selected_rows = []

for main_venue in merged_df['main_venue'].unique():
    venue_df = merged_df[merged_df['main_venue'] == main_venue].copy()

    if len(venue_df) < 2:
        continue

    # Sort by custom diversity score
    venue_df = venue_df.sort_values(by='score_diversity', ascending=True).reset_index(drop=True)

    # Filter out rows where actionability, grounding_specificity, and helpfulness are all the same
    def is_uniform_main_scores(row):
        vals = [row[f"{aspect}_label"] for aspect in ['actionability', 'grounding_specificity', 'helpfulness']]
        return len(set(vals)) == 1

    venue_df['uniform_main_scores'] = venue_df.apply(is_uniform_main_scores, axis=1)

    # Try to find a low-diversity example that is NOT fully uniform
    non_uniform_df = venue_df[~venue_df['uniform_main_scores']]
    
    if not non_uniform_df.empty:
        low_div_row = non_uniform_df.iloc[0]
    else:
        # Fallback: allow a uniform one if necessary
        low_div_row = venue_df.iloc[0]

    # High diversity example (last row)
    high_div_row = venue_df.iloc[-1]
    if high_div_row['review_point'] == low_div_row['review_point'] and len(venue_df) > 1:
        high_div_row = venue_df.iloc[-2]

    # Final check: only keep if they're different
    if low_div_row['review_point'] != high_div_row['review_point']:
        selected_rows.extend([low_div_row, high_div_row])


# Step 8: Final output
summary_columns = ['main_venue', 'venue', 'review_point']
for aspect in aspects:
    summary_columns += [f"{aspect}_label", f"chatgpt_{aspect}_score"]

summary_df = pd.DataFrame(selected_rows)[summary_columns]

# Output
summary_df
# summary_df.to_csv("main_venue_uniform_and_diverse_examples.csv", index=False)


,main_venue,venue,review_point,actionability_label,chatgpt_actionability_score,grounding_specificity_label,chatgpt_grounding_specificity_score,verifiability_label,chatgpt_verifiability_score,helpfulness_label,chatgpt_helpfulness_score
0,ACL,ACL_2017,- In section 2.3 the authors use Lample et al....,5,4,5,5,5,X,5,4
2,ACL,ACL_2017,- The concept of energy is mentioned for the f...,5,5,5,5,5,3,5,4
13,ICLR,ICLR_2022,"2 The technical contribution is limited, i.e.,...",1,1,2,3,2,2,2,1
14,ICLR,ICLR_2021,"4. In Section 4.1, \epsilon is not used in equ...",5,5,5,5,0,4,5,5
21,NIPS,NIPS_2018,- The paper opens that learning long-range dep...,2,2,5,5,4,3,3,3
24,NIPS,NIPS_2016,"2. On line 205, it should be Fig. 1 instead of...",5,5,5,5,0,X,5,5
0,ARR,ARR_2022,- Table 4 and 5 would be more readable if they...,5,5,5,5,5,1,5,5
2,ARR,ARR_2022,"6. Lines 170 to 171, “unreliable neighbors” an...",5,5,5,5,0,X,5,4


In [26]:
summary_df
# Convert the summary_df DataFrame into a LaTeX table
latex_table = summary_df.to_latex(index=False, caption="Summary of Selected Rows", label="tab:summary_table")
print(latex_table)

\begin{table}
\caption{Summary of Selected Rows}
\label{tab:summary_table}
\begin{tabular}{lllrrrrrlrr}
\toprule
main_venue & venue & review_point & actionability_label & chatgpt_actionability_score & grounding_specificity_label & chatgpt_grounding_specificity_score & verifiability_label & chatgpt_verifiability_score & helpfulness_label & chatgpt_helpfulness_score \\
\midrule
ACL & ACL_2017 & - In section 2.3 the authors use Lample et al. Bi-LSTM-CRF model, it might be beneficial to add that the input is word embeddings (similarly to Lample et al.) - Figure 3, KNs in source language or in English? ( since the mentions have been translated to English). In the authors' response, the authors stated that they will correct the figure. & 5 & 4 & 5 & 5 & 5 & X & 5 & 4 \\
ACL & ACL_2017 & - The concept of energy is mentioned for the first time in Section 3.1. Even though the explanation provided is enough at that point, it would be nice to refresh the idea of energy in Section 5.2 (where it is